In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.support.ui import Select

import requests
from bs4 import BeautifulSoup
import re as re
import time
import pandas as pd
import numpy as np


import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords

from wordcloud import WordCloud
from langdetect import detect
from textblob import TextBlob
import spacy

In [2]:
def get_aljazeera():
    
    driver = webdriver.Chrome()
    url= 'https://www.aljazeera.com/'
    driver.get(url)
    time.sleep(10)
    cookies_accept = driver.find_element(By.ID, "onetrust-accept-btn-handler").click()
    ukraine_button = driver.find_element(By.PARTIAL_LINK_TEXT, 'Ukraine war').click()
    time.sleep(10)
    lnks=driver.find_elements(By.TAG_NAME,"a")
    lst_aljazeera=[lnk.get_attribute('href') for lnk in lnks]
    lst_aj=[url for url in lst_aljazeera if '/news/2' in url ]
    while len(lst_aj)<100:
        show_more = driver.find_element(By.CLASS_NAME, "show-more-button")
        driver.execute_script("arguments[0].scrollIntoView();",show_more)
        show_more= driver.find_element(By.CLASS_NAME, "show-more-button").click()
        time.sleep(3)
        lnks=driver.find_elements(By.TAG_NAME,"a")
        lst_aljazeera=[lnk.get_attribute('href') for lnk in lnks]
        lst_aj=[url for url in lst_aljazeera if '/news/2' in url ]
        if len(lst_aj)<100:
            continue
    return lst_aj

In [3]:
def create_dict_aj(lst_aj):
    dict_aj={}
    for url in lst_aj:
        html = requests.get(url)
        soup = BeautifulSoup(html.content, "html.parser")
        article= soup.getText()
        article = article[article.find('facebooktwitterwhatsapp'):]
        article = article[:article.find('Source: Al Jazeera')]
        article = article.replace('\n','').replace('\xa0',' ').replace('facebooktwitterwhatsapp','')
        dict_aj[url]= article
    return dict_aj

In [4]:
def descriptives(dict_aj):
    lst_polarity = [round(TextBlob(article).sentiment.polarity,5) for article in dict_aj.values()]
    lst_subjectivity = [round(TextBlob(article).sentiment.subjectivity,5) for article in dict_aj.values()]
    length=[len(articles) for articles in dict_aj.values()]
    ukraine=[articles.lower().count('ukraine') for articles in dict_aj.values()]
    war=[articles.lower().count('war') for articles in dict_aj.values()]
    russia=[articles.lower().count('russia') for articles in dict_aj.values()]
    putin = [articles.lower().count('putin') for articles in dict_aj.values()]
    zelensky= [articles.lower().count('zelensky') for articles in dict_aj.values()]
    dict_length= {'polarity':lst_polarity, 'subjectivity':lst_subjectivity,'length':length,'ukraine':ukraine,'war':war, 'russia':russia,'putin':putin,'zelensky':zelensky}
    df_aj_descr= pd.DataFrame(dict_length)
    return df_aj_descr

In [5]:
def create_df_aj(dict_aj):
    idx= {'article'}
    df_aj= pd.DataFrame(dict_aj,idx).transpose().reset_index()
    df_aj= df_aj.rename(columns={"index": "link"})
    return df_aj

In [6]:
lst_aj = get_aljazeera()

In [7]:
dict_aj = create_dict_aj(lst_aj)

In [8]:
df_aj_descr= descriptives(dict_aj)

In [9]:
df_aj= create_df_aj(dict_aj)

In [10]:
df_aj_all =df_aj.join(df_aj_descr)

In [15]:
df_aj_all

,link,article,polarity,subjectivity,length,ukraine,war,russia,putin,zelensky
0,https://www.aljazeera.com/news/2023/2/26/russi...,"Here is the situation as it stands on Sunday, ...",0.08643,0.36405,1861,5,4,11,0,0
1,https://www.aljazeera.com/news/2023/2/26/macro...,French President Emmanuel Macron has said he w...,0.06972,0.35659,5037,9,6,11,4,2
2,https://www.aljazeera.com/news/2023/2/25/thous...,Protesters in Paris and Berlin capitals have r...,0.05854,0.35858,3666,11,3,9,1,0
3,https://www.aljazeera.com/news/2023/2/25/g20-m...,Finance chiefs of the world’s largest economie...,0.05854,0.37677,5345,5,4,9,0,0
4,https://www.aljazeera.com/news/2023/2/25/ukrai...,Ukraine has issued a postage stamp based on a ...,0.17239,0.30488,1887,3,0,5,2,0
...,...,...,...,...,...,...,...,...,...,...
99,https://www.aljazeera.com/news/2023/2/8/blinke...,US Secretary of State Antony Blinken has met w...,0.02982,0.35126,2883,6,4,4,1,0
100,https://www.aljazeera.com/news/2023/2/8/us-has...,The United States has questions to answer over...,0.00488,0.27061,3708,1,0,8,1,0
101,https://www.aljazeera.com/news/2023/2/8/ukrain...,Ukrainian President Volodymyr Zelenskyy has ur...,0.08636,0.31209,4282,10,6,9,0,8
102,https://www.aljazeera.com/news/2023/2/8/russia...,Here is the situation as it stands on Wednesda...,-0.06573,0.21966,2159,6,3,5,0,0
